## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
# from sendgrid.helpers.mail import Mail, Email, To, Content
import brevo_python
import asyncio



In [3]:
load_dotenv(override=True)

True

In [6]:
import os
import brevo_python
from brevo_python.api import transactional_emails_api
from brevo_python.models.send_smtp_email import SendSmtpEmail
from brevo_python.models.send_smtp_email_to import SendSmtpEmailTo
from brevo_python.models.send_smtp_email_sender import SendSmtpEmailSender

def send_test_email_with_brevo():
    # Configure the Brevo API client
    configuration = brevo_python.Configuration()
    # Ensure you have set the BREVO_API_KEY environment variable
    api_key = os.environ.get('BREVO_API_KEY')
    if not api_key:
        raise ValueError("BREVO_API_KEY environment variable not set.")
    configuration.api_key['api-key'] = api_key

    # Create an instance of the API class
    api_instance = transactional_emails_api.TransactionalEmailsApi(brevo_python.ApiClient(configuration))

    # Define the sender
    # NOTE: This email MUST be a verified sender in your Brevo account.
    sender = SendSmtpEmailSender(email="oliver@oliverdreger.cloud", name="Oliver Dreger")

    # Define the recipient(s)
    # The 'to' field is a list, so you can add multiple recipients
    to = [SendSmtpEmailTo(email="oliver.dreger@gmail.com", name="Oliver Dreger")]

    # Create the email object
    send_smtp_email = SendSmtpEmail(
        sender=sender,
        to=to,
        subject="Test email from Brevo",
        text_content="This is an important test email sent via the Brevo API."
        # You can also use 'html_content' for HTML emails
        # html_content="<html><body><h1>This is a test</h1></body></html>"
    )

    # Send the email
    try:
        api_response = api_instance.send_transac_email(send_smtp_email)
        print("Email sent successfully!")
        print(f"Response: {api_response}")
        # In contrast to SendGrid's status code, a successful call here
        # returns a result object. A failure will raise an exception.
    except brevo_python.ApiException as e:
        print(f"Exception when calling TransactionalEmailsApi->send_transac_email: {e}\n")

# Run the function to send the email
send_test_email_with_brevo()

Email sent successfully!
Response: {'message_id': '<202507220057.21181128372@smtp-relay.mailin.fr>',
 'message_ids': None}


In [ ]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

### Did you receive the test email

If you get a 202, then you're good to go!

If not, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with something to simply write to a flat file)

## Step 1: Agent workflow

In [3]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [4]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


In [7]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [9]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [ ]:
sales_agent1

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [11]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [ ]:
# Let's look at it
send_email

### And you can also convert an Agent into a tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

## And now it's time for our Sales Manager - our planning agent

In [15]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [16]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [18]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [19]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
tools

In [21]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [24]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.

--- 

## Transcript Summary

Here is a summary of the transcript for "Day 2 - Build AI Sales Agents with SendGrid: Tools & Collaboration in Agent SDK".

### Project Goal & Overview

The session's objective is to build an AI Sales Development Rep using the OpenAI Agents SDK. This will be done by constructing three distinct layers of agentic architecture:
1.  **Simple Workflow:** Manually orchestrating basic agent calls.
2.  **Tool Use:** Creating an agent that can use an external tool.
3.  **Agent Collaboration:** Exploring two methods for agents to call each other: treating agents as tools and using "handoffs."

### Prerequisites: Setting up SendGrid

Before coding, the project requires setting up a free tool called **SendGrid** (owned by Twilio) to enable the agents to send emails.

**Setup Steps:**
*   **Create a Free Account:** Visit the SendGrid website and register.
*   **Generate an API Key:** Navigate to **Settings -> API Keys**, create a new key, and copy it.
*   **Verify Sender Identity:** Go to **Settings -> Sender Authentication** and use the "verify a single sender" option to confirm ownership of your email address. This authorizes SendGrid to send emails *from* that address for experimental purposes.
*   **Update Environment Variables:** Add the copied key to your `.env` file with the variable name `SendGrid API key`.

### Phase 1: Simple Agent Workflow Implementation

The first part of the code demonstrates a basic workflow by creating and running three distinct sales agents for a fictional SOC 2 compliance company named "Comply AI."

*   **Defining Agent Personas:** Three different system prompts ("instructions") are created to define the agents' personalities:
    *   **Agent 1:** Professional and serious.
    *   **Agent 2:** Humorous and engaging.
    *   **Agent 3:** Busy, concise, and to-the-point.
*   **Creating the Agents:** Three agent objects are instantiated using the `gpt-4-mini` model, each assigned one of the distinct personas.

### Demonstrating a New Technique: Streaming Output

The transcript introduces a more advanced way to receive output from an agent in real-time.
*   **Method:** Instead of the standard `runner.run()`, the code uses `runner.run_streamed()`.
*   **How it Works:** This method returns a **coroutine** rather than a complete response. It must be used with a special `async for` loop to process the text as it streams back token-by-token.
*   **Result:** Running the "professional" agent with this method successfully demonstrates a live, streaming output, showing the agent generating a high-quality, professional sales email. This confirms the LLM's proficiency at this task and shows how to create a more interactive user experience.

--- 
## Transcript Summary 

--- 

### Introduction and Project Goal

*   **Context:** This is the beginning of Day 2, Week 2. The session is hands-on and focused on building the first project using the OpenAI Agents SDK.
*   **Project:** The goal is to build a Sales Development Rep (SDR) agent.
*   **Layered Approach:** The project will be constructed in three distinct layers of increasing complexity:
    1.  A simple workflow of agent calls, orchestrated manually.
    2.  An agent that can use a tool (contrasting with the manual JSON method from Week 1).
    3.  Agents that collaborate with other agents, which will be achieved in two different ways: by treating agents as tools, and by using "handoffs."

### Tool Setup: SendGrid

*   **Tool Introduction:** The lab begins by setting up a tool called **SendGrid**.
    *   SendGrid is a service for sending transactional emails. It is owned by Twilio and is similar to services like SparkPost.
    *   It is free to start.
*   **Setup Instructions:** A precise, step-by-step guide is provided for setting up a SendGrid account:
    1.  Visit the SendGrid website and sign up for a free account.
    2.  Once logged in, navigate to **Settings -> API Keys**.
    3.  Create a new API key and copy it to your clipboard.
    4.  Navigate to **Settings -> Sender Authentication**.
    5.  Click "Verify a single sender" and follow the process to verify ownership of your personal email address. This allows you to send emails *from* that address.
*   **Project Scope:** The speaker clarifies that this setup is for experimental purposes only; the project will only involve sending emails back to yourself, not actual cold emails to others.
*   **Environment Configuration:** The final step is to edit the `.env` file and add a new line: `SENDGRID_API_KEY=` followed by the copied API key.

### Agent Creation and Workflow

*   **Imports:** The necessary libraries are imported, including `agents`, `FunctionTool`, a helper for streaming text, and SendGrid-specific modules.
*   **Defining Agent Personas:** The first part of the project involves a simple, manually orchestrated workflow. Three distinct agent instructions (system prompts) are created for a fictional SOC 2 compliance company called "ComplyAI":
    1.  **Professional Agent:** Writes "professional, serious" cold emails.
    2.  **Humorous Agent:** Writes "witty, engaging" cold emails designed to get a response.
    3.  **Concise Agent:** Writes "concise, to the point" emails, adopting the persona of a busy sales agent.
*   **Instantiating Agents:** Three agent objects are created (`sales_agent_one`, `two`, `three`), each initialized with its unique name, its corresponding instruction set, and the `gpt-4o-mini` model to keep costs low.

### Running an Agent with Streaming

*   **Introducing Streaming:** A new method for running agents is introduced: `runner.run_streamed()`. This allows the agent's response to be streamed back bit by bit, rather than waiting for the full response.
*   **Technical Details:**
    *   The `runner.run_streamed()` call does not use `await`, which means it returns a **coroutine** object, not an immediate result.
    *   To process this coroutine, a special `async for` loop is used to iterate through the streamed data chunks.
    *   Boilerplate code inside the loop checks if a chunk is text and prints it to the console without creating a new line for each chunk, resulting in a smooth, continuous output.
*   **Live Demonstration:** The code is run for the first agent (the professional one). As expected, the response streams back to the console. The resulting email is high-quality, professional, and well-suited for its purpose, demonstrating that LLMs are very effective at this kind of task.

--- 

### Implementing Concurrent Agent Execution with `asyncio`

*   **Goal:** To run multiple agents simultaneously to generate different versions of a sales email.
*   **Method:** The `asyncio.gather()` function is used. This is a key feature for asynchronous programming in Python.
*   **Technical Explanation:**
    *   The speaker clarifies that `asyncio` is not true multithreading (where the CPU time-slices). Instead, it uses an **event loop**.
    *   When one agent's process is paused waiting for I/O (like an API response from OpenAI), the event loop switches to run another agent's process.
    *   Since most of the time is spent waiting on I/O, this allows all three agents to effectively run in parallel, significantly speeding up the process.
*   **Implementation:**
    *   The `runner.run()` method is called for each of the three sales agents.
    *   These three function calls are wrapped inside `asyncio.gather()`.
    *   The entire block is awaited, which collects the results from all three parallel runs once they complete.
    *   The results are then printed, showing three different emails, with the third one being notably concise as per its instructions.

### Creating a "Picker" Agent Workflow

*   **Next Step:** A new agent, named `sales_picker`, is created to act as a final evaluation step.
*   **Agent's Prompt/Instructions:**
    *   Its task is to pick the "best" cold sales email from the options provided.
    *   It's instructed to act like a customer and choose the email it would be most likely to respond to.
    *   A classic prompting technique is used: a final constraint is added, telling it, "Don't give an explanation. Reply with the email you select only."
*   **Workflow Orchestration:**
    1.  The three sales agents are run in parallel using `asyncio.gather()` to generate three email drafts.
    2.  The collected outputs from the three agents are passed to the `sales_picker` agent.
    3.  This entire multi-step process is wrapped in a single **trace**, allowing for unified observability.
*   **Result & Tracing:** The `sales_picker` returns a single email. The speaker then clicks a link to view the trace in the OpenAI platform, which visualizes the entire workflow: the parallel calls to the three sales agents followed by the single call to the picker agent, demonstrating clear and elegant observability.

### Transitioning to Agent Tool Use

*   **Recap and Critique of Week 1:** The speaker contrasts the current elegant workflow with the manual process from Week 1.
    *   In Week 1, creating a tool required writing "chunky," "boilerplate" JSON to describe the function, its parameters, etc.
    *   It also required writing a `handle_tool_calls` function with "hokey" `if` statements (or fancy but equivalent `globals()` lookups) to route the tool calls. This is described as "gunk."
*   **The New, Simplified Approach:** The Agents SDK drastically simplifies this entire process without sacrificing flexibility.

### Defining a Tool with the Agents SDK

*   **The Function:** A new Python function, `send_email(email_body)`, is defined.
    *   It contains boilerplate code to send an email using the SendGrid API.
    *   **Crucially, it includes a docstring:** `"Send out an email with the given body to all sales prospects."`
    *   The function has placeholder emails for the "from" and "to" addresses. The "from" email must be the one verified in SendGrid. The "to" email should be changed by the user to avoid flooding the speaker's inbox.
*   **The Magic Decorator:** Instead of writing JSON, the **`@function_tool` decorator** is placed directly above the `send_email` function definition.
*   **What the Decorator Does:**
    *   It automatically converts the Python function into a `FunctionTool` object.
    *   The object's `description` is automatically populated from the function's **docstring**.
    *   It introspects the function's signature and automatically generates the entire complex **`params_json_schema`** (the boilerplate JSON from Week 1) on the user's behalf.
*   **Conclusion:** This demonstrates the power of a good framework. It removes all the tedious boilerplate work (`faffing around with JSON objects`) while leaving the developer in full control of the core logic.

--- 

### The Core Concept: Turning Agents into Tools

*   **The New Idea:** The speaker introduces a "confusing" but powerful concept: in addition to turning a Python function into a tool, you can also turn an **entire agent** into a tool.
*   **What it Means:**
    *   An agent, which is a process that calls an LLM with a specific prompt (like the "sales agent"), can be packaged and treated as a single, callable tool.
    *   This creates a **wrapper** around the agent. When this new "agent tool" is called, it triggers the underlying agent to execute its LLM call.
*   **How to Do It:** The process is surprisingly simple. You just call the `.as_tool()` method on an existing agent object.

### The Implementation: Creating Agent-Tools

*   **Demonstration:**
    *   The speaker takes `sales_agent_one` and calls `sales_agent_one.as_tool()`.
    *   This method requires a `tool_name` and a `description` for the new tool.
    *   The result is a `FunctionTool` object, identical in structure to the one created from a simple Python function earlier. It has a name, a description, and the automatically generated "JSON gunk" (the `params_json_schema`).
    *   The key difference is that its internal function, when called, will execute the agent's run.
*   **Creating Multiple Tools:**
    *   The speaker explicitly (without a loop, for clarity) converts all three sales agents (`sales_agent_one`, `two`, and `three`) into three corresponding tools (`tool_one`, `two`, and `three`).
    *   A final list, also named `tools`, is created. It contains **four** items: the three agent-as-tools, and the original `send_email` function-as-a-tool.

### Creating a "Sales Manager" Planning Agent

*   **The Goal:** To create a higher-level "planning agent" that orchestrates the other tools. This moves beyond a simple, hardcoded Python script to an agent that makes its own decisions.
*   **Agent's Prompt/Instructions:**
    *   **Role:** "You are a sales manager working for ComplyAI."
    *   **Core Directive:** "You use the tools given to you to generate cold sales emails. You never generate sales emails yourself."
    *   **Process:** "You try all three tools once before choosing the best."
    *   **Final Action:** "You pick the single best email and use the Send Email tool to send the best email and only the best email to the user."
*   **Instantiation and Execution:**
    *   A new `sales_manager_agent` is created.
    *   It is initialized with its instructions, the list of all **four** tools, and a model.
    *   The agent is run with the simple prompt: "send a cold sales email addressed to dear CEO."

### Results and Tracing

*   **Execution:** The run completes quickly (18 seconds).
*   **Outcome 1: The Email:** The speaker confirms they received an email.
    *   It's a well-formed sales email.
    *   It correctly came from the verified sender address.
    *   It intelligently replaced "dear CEO" with a placeholder like `[CEO's Name]`, showing a degree of templating awareness. An exercise to integrate proper mail merge is mentioned.
*   **Outcome 2: The Trace:** The speaker examines the execution trace in the OpenAI platform.
    *   The trace shows the `sales_manager` agent called four tools in sequence: `Sales Agent 1`, `Sales Agent 2`, `Sales Agent 3`, and finally `Send Email`.
    *   Crucially, the trace visualizes the hierarchy. It shows that the calls to the "sales agent" tools were wrappers that, **underneath**, contained another agent call (e.g., the "Professional Sales Agent").
    *   In contrast, the `send_email` tool call shows it was just a direct function call with a body parameter.
*   **Conclusion:** This demonstrates a hierarchical system where a manager agent can decide the order in which to use other agents (wrapped as tools) to complete a complex task. The user is encouraged to study the trace to fully understand these interactions.

--- 

### Recap of Agent Architectures

The speaker begins by summarizing the architectures built so far:
1.  **Simple Workflow:** Manually orchestrated agents in Python code, using `asyncio.gather` to run them in parallel, followed by a final "picker" agent.
2.  **Agents with Tools:**
    *   Wrapped a simple Python function (`send_email`) into a tool.
    *   Wrapped entire agents (the three email writers) into tools using the `.as_tool()` method.
    *   Provided all these tools to a "sales manager" agent, which decided which tools to use and when.

---

### Introducing the "Handoff" Concept

This section introduces a new, similar, yet distinct construct for agent collaboration called a **handoff**.

*   **Similarity to "Agent as Tool":** A handoff is a way for one agent to delegate tasks to another agent, which sounds very similar to wrapping an agent as a tool. An agent can be given a list of tools and a list of handoffs.
*   **The Conceptual Difference:**
    *   **Agent as Tool:** Think of this as an agent using a small, self-contained feature to help with its main job. It's like using a calculator.
    *   **Handoff:** This is more like full delegation. The primary agent is giving complete responsibility and ownership of a specialized task to another agent. It's passing an entire job, not just using a feature.
*   **The Key Technical Difference:**
    *   **Agent as Tool:** This is a **request-response** model. The main agent calls the tool, waits for a response, and then **control passes back** to the main agent to continue its execution.
    *   **Handoff:** This is a **one-way transfer of control**. The main agent does its part and then passes control to the other agent. The execution flow **does not come back** to the original agent.

---

### Building the Components for a Handoff Workflow

To demonstrate this, a new workflow is constructed with several new agents and tools.

1.  **New Agent: `subject_writer`**
    *   **Purpose:** To write a compelling subject line for a given email body.
    *   **Implementation:** It's created as an agent and then immediately converted into a **tool** using `.as_tool()`. This is because writing a subject is framed as a small, discrete task.

2.  **New Agent: `html_converter`**
    *   **Purpose:** To convert a plain-text email (which may contain Markdown) into a well-formatted HTML email.
    *   **Implementation:** Like the subject writer, this agent is also converted into a **tool**.

3.  **New Tool: `send_html_email`**
    *   **Purpose:** A new version of the send email function that now accepts a `subject` and a `body` and sends the email as HTML.
    *   **Implementation:** This is a standard Python function with an `@function_tool` decorator. It requires the user to update their verified sender email and the recipient email.

---

### Creating the Handoff Agent

Now, the agent that will *receive* the handoff is created.

*   **Agent Name:** `emailer_agent`
*   **Purpose:** This agent's job is to format and send the final email.
*   **Instructions (The agent's internal plan):**
    1.  "You receive the body of an email."
    2.  "You first use the `subject_writer` tool."
    3.  "Then [you use] the `html_converter` tool."
    4.  "Finally, you send the email."
*   **Key New Property: `handoff_description`**
    *   This agent is initialized with a `handoff_description`: `"Convert an email to HTML and send it."`
    *   This description is how the agent "announces itself" to other agents that might want to hand off to it. It's analogous to a tool's description.

### Assembling the Final Structure

The speaker clarifies the final setup before running it:
*   **`tools` List:** This will contain the three original sales agents, each wrapped as a tool. These agents will generate the email content.
*   **`handoffs` List:** This will contain the single `emailer_agent`.
*   **The `emailer_agent`'s Internal Tools:** It's highlighted that the `emailer_agent` itself has its own list of three tools: the `subject_writer` tool, the `html_converter` tool, and the `send_html_email` tool.

The user is encouraged to print these lists and objects to fully understand the nested, hierarchical structure before the final execution.

--- 

### The Final Workflow Execution

*   **Moment of Truth:** The speaker runs the final, most complex agentic system.
*   **The "Sales Manager" Agent's Instructions (Recap):**
    *   **Role:** Sales Manager for "ComplyAI".
    *   **Core Directive:** Must use tools to generate emails; never writes them itself.
    *   **Process:** Try all three sales email agent-tools at least once. It can try them multiple times if the first results are unsatisfactory.
    *   **Selection:** Use its own judgment to select the single best email.
    *   **Final Action (The Handoff):** After picking the best email, it must **hand off** to the `emailer_agent` to format and send it.
*   **The Crux of the Architecture:** The agent is initialized with both a list of `tools` (for request-response tasks) and a list of `handoffs` (for one-way delegation of control). This separation is the key to the design.
*   **Execution and Result:** The process runs for about a minute. The final output is a short, crisp email that was selected by the manager.

### Analyzing the Trace: Visualizing the Complex Workflow

*   **The Trace:** The speaker examines the execution trace, named "Automated SDR," which reveals the agent's decision-making process.
*   **Key Observations:**
    1.  The `sales_manager` first called each of the three sales agents once.
    2.  It then decided to call them all a **second time**, demonstrating its autonomy to try again if not satisfied.
    3.  After the six tool calls, the trace clearly shows a **handoff** to the `emailer_agent`.
    4.  The `emailer_agent` then took control and executed its own internal sequence of tool calls: `subject_writer`, `html_converter`, and `send_email`.
    5.  Crucially, the trace shows that after the handoff, control **did not return** to the original `sales_manager` agent. The `emailer_agent` completed the workflow.

### Conclusion and Key Takeaways

*   **Summary of Concepts:** The session covered orchestrating agents with simple Python, using tools for functions, wrapping agents as tools, and finally, using handoffs for delegation.
*   **Main Takeaway:** The OpenAI Agents SDK is relatively simple and lightweight, yet it allows for the creation of complex and advanced agentic workflows in a short amount of time.

### Exercises for the User

The speaker provides several exercises to reinforce the learning.

1.  **Identify Design Patterns:** Go back through the code and identify the specific agentic design patterns that were used. Pinpoint the exact moment the system transitioned from a simple "agent workflow" to a true "agentic agent" with autonomy.
2.  **Extend the System:** Add more tools and agents. The system is designed to be easily extensible. The speaker notes that calling it an "SDR" is a stretch; it's currently just a cold email writer, but it could be built into something more interactive.
3.  **Hard Challenge (Long-Lived Agent):** For those with engineering experience, figure out how to handle replies to the emails. This would involve:
    *   Researching how to use **webhooks** with SendGrid to receive replies.
    *   Triggering the agent workflow based on an incoming email.
    *   Managing conversation state to identify the original context.
    *   This is presented as more of an engineering challenge than an agentic one.

### Commercial Applications

*   **Direct Application:** The project itself could be the basis for a sales automation tool that generates email campaigns and even engages in conversations (if the hard challenge is completed).
*   **General Applicability:** The core point is that this approach can be applied to automate **any end-to-end business process**.
*   **Other Examples:** The speaker suggests other contexts, such as their own day job in **recruitment** at Nebula, which involves similar large-scale outreach activities.
*   **Final Thought:** The user is encouraged to think of any business area with high-volume, repeatable activities and imagine how a system of collaborating agents with autonomy, tools, and handoffs could automate that complex process.

--- 

### Call for Community Contributions

*   The speaker encourages students who build the more advanced, interactive version of the sales outreach agent (the "hard challenge" that can handle email replies) to share their work.
*   **How to Share:**
    1.  **Community Contributions Folder:** There is a specific folder in the week two materials for community contributions. Students should submit their code there via a Pull Request (PR), with instructions available in the class resources. This allows the speaker and other students to see the work.
    2.  **LinkedIn:** The speaker recommends the "great trick" of posting projects on LinkedIn and tagging the speaker. This helps amplify the student's work, allows the speaker to provide feedback, and shows a wider audience how agentic AI is being applied to business problems.

### Preview of the Next Session

*   After congratulating the students on completing the project and learning about the OpenAI Agents SDK, the speaker provides a preview of the next lecture.
*   **Topics to be Covered:**
    1.  **Revisiting Tools vs. Agents:** The discussion on the distinctions between tools and agents will be revisited one more time.
    2.  **Guardrails:** The next session will introduce the "super important" topic of guardrails, which are methods for putting controls around what the agents are doing.
    3.  **New Larger Project:** A larger project will be started.